# mosi_reduce_video_features

Dans ce notebook :  

- On récupère pour chaque vidéo un gros CSV OpenFace (landmarks, pose, gaze, AUs…).
- On essaie d’abord de garder uniquement les colonnes AU si elles existent (meilleure info émotionnelle).
- Sinon, on garde les colonnes de pose et de regard (pose_Tx, gaze_0_x…).
- Sinon, on garde les 20 premières colonnes numériques du CSV.
- On limite le CSV final à 50 frames maximum.
- On sauvegarde ce CSV réduit, qui sera ensuite transformé en video_feat.

Ces CSV réduits sont ensuite convertis en tenseurs video_feat dans mosi_prepared_dataset.


In [1]:
import os
import pandas as pd
import numpy as np

VIDEO_IN = "MOSI_full/features_video"             # CSV OpenFace originaux
VIDEO_OUT = "MOSI_full/features_video_reduced"    # CSV réduits

os.makedirs(VIDEO_OUT, exist_ok=True)

MAX_FRAMES = 50

# Colonnes AU idéales (si l’OpenFace les a produites)
AU_COLS = [
    "AU01_r", "AU02_r", "AU04_r", "AU06_r",
    "AU12_r", "AU15_r", "AU20_r", "AU26_r"
]

# Colonnes alternatives si pas d’AU
ALT_COLS = [
    "pose_Tx", "pose_Ty", "pose_Tz",
    "pose_Rx", "pose_Ry", "pose_Rz",
    "gaze_0_x", "gaze_0_y", "gaze_0_z",
    "gaze_1_x", "gaze_1_y", "gaze_1_z"
]


In [2]:
def reduce_video_csv(in_path, out_path):
    df = pd.read_csv(in_path)
    df = df.rename(columns={c: c.strip() for c in df.columns})

    # 1) Essayer d'utiliser les AU
    available_aus = [c for c in AU_COLS if c in df.columns]

    if len(available_aus) > 0:
        df_red = df[available_aus]
        print(f"✔ AUs trouvés pour {os.path.basename(in_path)} ({len(available_aus)} colonnes)")
    else:
        # 2) Essayer colonnes alternatives (pose/gaze)
        available_alt = [c for c in ALT_COLS if c in df.columns]

        if len(available_alt) > 0:
            df_red = df[available_alt]
            print(f"✔ Colonnes alternatives trouvées ({len(available_alt)} colonnes) : {os.path.basename(in_path)}")
        else:
            # 3) Sinon fallback aux 20 premières colonnes numériques
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            df_red = df[numeric_cols[:20]]
            print(f"⚠ Aucun AU ni ALT — utilisation des 20 premières colonnes numériques ({len(df_red.columns)} colonnes).")

    # Limiter à MAX_FRAMES
    df_red = df_red.iloc[:MAX_FRAMES]

    # Sauvegarde
    df_red.to_csv(out_path, index=False)


In [3]:
for fname in os.listdir(VIDEO_IN):
    if not fname.endswith(".csv"):
        continue

    in_path = os.path.join(VIDEO_IN, fname)
    out_path = os.path.join(VIDEO_OUT, fname)

    try:
        reduce_video_csv(in_path, out_path)
        print("Saved →", out_path)

    except Exception as e:
        print("❌ Erreur pour", fname, ":", e)


✔ Colonnes alternatives trouvées (12 colonnes) : 03bSnISJMiM.csv
Saved → MOSI_full/features_video_reduced\03bSnISJMiM.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 0h-zjBukYpk.csv
Saved → MOSI_full/features_video_reduced\0h-zjBukYpk.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 1DmNV9C1hbY.csv
Saved → MOSI_full/features_video_reduced\1DmNV9C1hbY.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 1iG0909rllw.csv
Saved → MOSI_full/features_video_reduced\1iG0909rllw.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 2iD-tVS8NPw.csv
Saved → MOSI_full/features_video_reduced\2iD-tVS8NPw.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 2WGyTLYerpo.csv
Saved → MOSI_full/features_video_reduced\2WGyTLYerpo.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 5W7Z1C_fDaE.csv
Saved → MOSI_full/features_video_reduced\5W7Z1C_fDaE.csv
✔ Colonnes alternatives trouvées (12 colonnes) : 6Egk_28TtTM.csv
Saved → MOSI_full/features_video_reduced\6Egk_28TtTM.csv
✔ Colonnes alternatives 